# Logging training progess using tensorboard

quantnn now has some limited functionality to log training progress using [Tensorflow tensorboard](https://www.tensorflow.org/tensorboard/). This notebook provides an example how this functionality can be used using the convolutional rain rate retrieval form [this notebook](https://github.com/simonpf/quantnn/blob/main/notebooks/convolutional_rain_rate_retrieval.ipynb).

> **Note**: This is still very new functionality so specific details of the API may change in the future.


In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from quantnn.plotting import set_style
set_style()


Bad key "mathtext.fallback" on line 203 in
/home/simonpf/src/quantnn/quantnn/data/matplotlib_style.rc.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


## Training setup

To setup the example, we load the training and validation data and define a very simple convolutional network.

In [2]:
from quantnn.examples.gprof_conv import download_data
download_data()

In [3]:
data = np.load("data/gprof_conv.npz")
x_train = data["x_train"]
y_train = data["y_train"]
x_val = data["x_val"]
y_val = data["y_val"]

In [4]:
import torch
from torch import nn
from quantnn.qrnn import QRNN

quantiles = np.linspace(0.01, 0.99, 99)

model_fc = nn.Sequential(
    nn.Conv2d(13, 128, 1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.Conv2d(128, 128, 1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.Conv2d(128, 128, 1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.Conv2d(128, quantiles.size, 1)
)

qrnn = QRNN(quantiles=quantiles, model=model_fc)

## Activating tensorboard logging

Tensor board loggin is activated by passing a ``quantnn.models.pytorch.logging.TensorBoardLogger`` object as the ``logger`` keyword argument to the QRNNs ``train`` method. 

The directory to which the logging data is written can be controlled using the ``log_directory`` argument passed to the ``TensorBoardLogger`` class upon initialization. If it is set to ``None``, logs for all experiments will be written into separate sub-folders in the ``run`` directory.

In [9]:
from quantnn.models.pytorch import BatchedDataset
from quantnn.models.pytorch.logging import TensorBoardLogger
from torch.optim import Adam

training_data = BatchedDataset((x_train, y_train), 4)
validation_data = BatchedDataset((x_val, y_val), 4)
n_epochs = 20

# Set this explicitly 
log_directory = None
logger = TensorBoardLogger(n_epochs,
                           log_directory=log_directory)
qrnn.train(training_data=training_data,
           validation_data=validation_data,
           n_epochs=n_epochs,
           mask=-1,
           device="gpu",
           logger=logger,
           optimizer=Adam(model_fc.parameters(), lr=0.01),
           metrics=["Calibration"]);

Output()

                                                       
                                                       
                    Training history                   
                                                       
    Epoch         Training loss       Validation loss  
───────────────────────────────────────────────────────
   #     LR           Total          Total

  1   0.010         0.053          0.047

Output()

  2   0.010         0.053          0.046

...

 18   0.000         0.053          0.046

Output()

 19   0.000         0.052          0.047

Output()

 20   0.000         0.053          0.046

## Tracking progress on a specific input

By default, the tensor board logger will log only training and validaiton error. Custom data can be logged using a callback hook (defined by the ``epoch_begin_callback`` attributed of the ``TensorBoardLogger`` class) that is called at the beginning of each epoch.


The ``epoch_begin_callback`` is expected to have the following signature:


````python
def epoch_begin_callback(model, writer, epoch_index):
````
where the arguments correspond to the following
- ``model``: The PyTorch model that is trained in its current state.
- ``writer``: The  [SummaryWriter](https://pytorch.org/docs/stable/tensorboard.html)  that is used to log data for the current training session.
                
- ``epoch_index``: The index (zero-based) of the current epoch.
        
The example below illustrates how this functionality can be used to track the prediction on a specific input at the beginning of each epoch:

In [6]:
x = x_val[:1]
y = y_val[:1]

def make_prediction(writer, model, epoch_index):
    """
    Predicts the mean precipitation rate on the first sample
    from the validation set.
    
    Args:
        writer: The SummaryWriter object that is used to log
             to the tensbor board.
        model: The model attributed of the qrnn object being
            trained.
        epoch_index: The index (zero-based) of the current
            epoch.
    """
    # Make prediction
    y_mean = qrnn.posterior_mean(x)
    # Store output using add_image function of SummaryWriter
    writer.add_image("predicted_rain_rate", y_mean, epoch_index)
    
    # Store reference imager using add_image function of
    # SummaryWriter. No need to store for every epoch.
    writer.add_image("reference_rain_rate", y, 0)

In [7]:
from quantnn.models.pytorch import BatchedDataset
from quantnn.models.pytorch.logging import TensorBoardLogger


training_data = BatchedDataset((x_train, y_train), 4)
validation_data = BatchedDataset((x_val, y_val), 4)
n_epochs = 5
logger = TensorBoardLogger(n_epochs,
                           log_directory=None,
                           epoch_begin_callback=make_prediction)
qrnn.train(training_data=training_data,
           validation_data=validation_data,
           n_epochs=n_epochs,
           mask=-1,
           device="gpu",
              logger=logger);

Output()

                                                       
                                                       
                    Training history                   
                                                       
    Epoch         Training loss       Validation loss  
───────────────────────────────────────────────────────
   #     LR           Total          Total

  1   0.010         0.053          0.047

Output()

  2   0.010         0.053          0.047

...

  3   0.010         0.053          0.047

Output()

  4   0.010         0.053          0.046

Output()

  5   0.010         0.053          0.046